In [1]:
from pyscipopt import Model

m = Model("demo")
x = m.addVar("x", lb=0, ub=10)
m.setObjective(x, "maximize")
m.addCons(x <= 5)
m.optimize()
print("Optimal value:", m.getObjVal())

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
Optimal value: 5.0
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.02
Solving Nodes      : 0
Primal Bound       : +5.00000000000000e+00 (2 solutions)
Dual Bound         : +5.00000000000000e+00
Gap                : 0.00 %


In [4]:
from pyscipopt import Model, quicksum
import numpy as np

# Number of cities
n = 5

# Generate a random symmetric distance matrix
np.random.seed(42)
dist = np.random.randint(10, 100, size=(n, n))
for i in range(n):
    dist[i, i] = 0
    for j in range(i+1, n):
        dist[j, i] = dist[i, j]

print("Distance matrix:\n", dist)

# Create SCIP model
model = Model("TSP_MTZ")

# Binary variables: x[i,j] = 1 if path goes from i -> j
x = {}
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

# MTZ ordering variables: u[i] gives position of city i in tour
u = {}
for i in range(n):
    u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

# Objective: minimize travel distance
model.setObjective(
    quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
    "minimize"
)

# Each city has exactly one outgoing edge
for i in range(n):
    model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)

# Each city has exactly one incoming edge
for j in range(n):
    model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

# MTZ subtour elimination constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

# Solve the model
model.optimize()

# Print optimal solution
if model.getStatus() == "optimal":
    print(f"\nOptimal tour length = {model.getObjVal():.2f}")

    # Extract tour edges
    tour = []
    current = 0
    visited = {0}
    for _ in range(n):
        for j in range(n):
            if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                tour.append((current, j))
                current = j
                break
    print("Tour edges:", tour)


Distance matrix:
 [[ 0 24 81 70 30]
 [24  0 84 84 97]
 [81 84  0 62 11]
 [70 84 62  0 73]
 [30 97 11 73  0]]
presolving:

Optimal tour length = 211.00(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
   (0.0s) running MILP presolver
   (0.2s) MILP presolver found nothing
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 10 upgd conss, 0 impls, 10 clqs
   (0.2s) probing cycle finished: starting next cycle
   (0.2s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.2s) no symmetry present
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 24 implications, 10 cliques
presolved problem has 24 variables (20 bin, 0 int, 0 impl, 4 cont) and 22 constraints
     10 constraints

In [9]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd
import random

def solve_tsp_mtz(n, dist, instance_id):
    """
    Solve TSP with MTZ constraints using SCIP.
    Returns objective value and edges in the optimal tour.
    """
    model = Model(f"TSP_MTZ_{instance_id}")
    
    # Decision variables
    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

    u = {}
    for i in range(n):
        u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

    # Objective
    model.setObjective(
        quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
        "minimize"
    )

    # Degree constraints
    for i in range(n):
        model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    for j in range(n):
        model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

    # MTZ constraints
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

    # Solve
    model.optimize()

    if model.getStatus() == "optimal":
        obj = model.getObjVal()
        # Extract tour edges
        edges = []
        current = 0
        visited = {0}
        for _ in range(n):
            for j in range(n):
                if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                    edges.append((current, j))
                    current = j
                    break
        return obj
    else:
        return None, None


def generate_instances(num_instances=100, min_cities=5, max_cities=10, seed=42):
    """
    Generate and solve multiple random TSP instances.
    Returns a pandas DataFrame with summary results.
    """
    random.seed(seed)
    np.random.seed(seed)

    results = []
    for inst in range(num_instances):
        n = random.randint(min_cities, max_cities)
        dist = np.random.randint(10, 100, size=(n, n))
        for i in range(n):
            dist[i, i] = 0
            for j in range(i+1, n):
                dist[j, i] = dist[i, j]

        obj = solve_tsp_mtz(n, dist, inst)
        if obj is not None:
            results.append({
                "instance": inst,
                "cities": n,
                "objective": obj
            })
            print(f"Instance {inst}: {n} cities, optimal cost = {obj:.2f}")
        else:
            print(f"Instance {inst}: solver failed")

    return pd.DataFrame(results)


# Example: generate 10 small TSP instances
df = generate_instances(num_instances=10, min_cities=5, max_cities=7)

print(df.head())


presolving:
Instance 0: 7 cities, optimal cost = 261.00
Instance 1: 5 cities, optimal cost = 196.00
Instance 2: 5 cities, optimal cost = 317.00
Instance 3: 7 cities, optimal cost = 158.00
(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 14 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 14 upgd conss, 0 impls, 14 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 60 implications, 14 cliques
presolved problem has 48 variables (42 bin, 0 int, 0 impl, 6 cont) 

 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 40 implications, 12 cliques
presolved problem has 35 variables (30 bin, 0 int, 0 impl, 5 cont) and 32 constraints
     12 constraints of type <setppc>
     20 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
p 0.0s|     1 |     0 |     0 |     - |   locks|   0 |  35 |  34 |  32 |   0 |  0 |   1 |   0 | 0.000000e+00 | 3.620000e+02 |    Inf | unknown
p 0.0s|     1 |     0 |     0 |     - | vbounds|   0 |  35 |  34 |  32 |   0 |  0 |   3 |   0 | 0.000000e+00 | 3.200000e+02 |    Inf | unknown
  0.0s|     1 |     0 |    17 |     - |  1202k |   0 |  35 |  34 |  32 |   0 |  0 |   3 |   0 | 2.093333e+02 | 3.200000e+02 |  52.87%| unknown
r 0.0s|     1 |     0 